# PreProPy: Data Preprocessing Made Simple

This notebook demonstrates the key features of the PreProPy package, which combines three essential data preprocessing tools:

1. **NullSense**: Intelligent handling of missing values
2. **DupliChecker**: Duplicate record detection and removal
3. **ScaleNPipe**: Feature scaling and model pipeline creation

Let's explore how to use these tools in your data science workflow.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Import PreProPy functions
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
from prepropy import handle_nulls, drop_duplicates, get_duplicate_stats, scale_pipeline, get_available_scalers

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('colorblind')
%matplotlib inline

## Loading Sample Data

First, let's load the sample dataset we generated earlier. This dataset has intentionally introduced missing values and duplicate records.

In [ ]:
# Load the sample dataset
df = pd.read_csv('../sample_data.csv')

# Display the first few rows
print(f"Dataset shape: {df.shape}")
df.head()

In [ ]:
# Check for missing values
missing_values = df.isna().sum()
print("Missing values per column:")
print(missing_values)
print(f"\nTotal missing values: {missing_values.sum()}")

# Visualize missing values
plt.figure(figsize=(10, 6))
sns.barplot(x=missing_values.index, y=missing_values.values)
plt.title('Missing Values by Column')
plt.xticks(rotation=45)
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## 1. NullSense: Handling Missing Values

NullSense intelligently fills missing values based on column types. For numeric columns, it uses statistical measures like mean or median, and for categorical columns, it uses the mode (most frequent value).

In [ ]:
# Demonstrate different strategies for handling missing values
strategies = ['auto', 'mean', 'median', 'mode', 'zero']

# Create a copy of the dataframe with selected columns for demonstration
demo_df = df[['age', 'income', 'education', 'satisfaction']].copy()

# Function to count missing values in a dataframe
def count_missing(df):
    return df.isna().sum().sum()

# Demonstrate different strategies
results = {}
for strategy in strategies:
    filled_df = handle_nulls(demo_df, strategy=strategy)
    results[strategy] = filled_df.copy()
    print(f"Strategy: {strategy}, Missing values after: {count_missing(filled_df)}")

    # For demonstration, show first 5 rows of two columns
    if strategy == 'auto':
        print("\nSample of filled values with 'auto' strategy:")
        print("\nOriginal:")
        print(demo_df[['age', 'education']].head(5))
        print("\nFilled:")
        print(filled_df[['age', 'education']].head(5))

In [ ]:
# Compare the distribution before and after filling missing values
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
sns.histplot(demo_df['age'].dropna(), kde=True)
plt.title('Age Distribution (Before)')

plt.subplot(2, 2, 2)
sns.histplot(results['auto']['age'], kde=True)
plt.title('Age Distribution (After Auto Strategy)')

plt.subplot(2, 2, 3)
sns.histplot(demo_df['income'].dropna(), kde=True)
plt.title('Income Distribution (Before)')

plt.subplot(2, 2, 4)
sns.histplot(results['auto']['income'], kde=True)
plt.title('Income Distribution (After Auto Strategy)')

plt.tight_layout()
plt.show()

### Using the Auto Strategy

The `auto` strategy is particularly useful because it applies different filling methods based on column type:
- For numeric columns: uses median (more robust to outliers)
- For categorical columns: uses mode (most frequent value)

Let's proceed with the auto-filled dataset for the rest of our analysis:

In [ ]:
# Use the auto strategy for our main dataset
df_filled = handle_nulls(df, strategy='auto')

# Confirm no missing values remain
print(f"Missing values after filling: {count_missing(df_filled)}")

## 2. DupliChecker: Handling Duplicate Records

DupliChecker helps identify and remove duplicate records from your dataset.

In [ ]:
# Get statistics about duplicates in the dataset
# We'll exclude 'id' since it's a unique identifier
duplicate_stats = get_duplicate_stats(df_filled, subset=df_filled.columns[1:])

print("Duplicate Statistics:")
for key, value in duplicate_stats.items():
    print(f"  {key}: {value}")

# Visualize duplicate percentage
plt.figure(figsize=(8, 6))
labels = ['Unique Records', 'Duplicate Records']
sizes = [duplicate_stats['unique_count'], duplicate_stats['duplicate_count']]
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.title('Duplicate Records in Dataset')
plt.show()

In [ ]:
# Demonstrate different approaches to handling duplicates
print("Original dataset shape:", df_filled.shape)

# Drop duplicates considering all non-id columns
df_no_dups = drop_duplicates(df_filled, subset=df_filled.columns[1:])
print("After dropping duplicates:", df_no_dups.shape)

# Drop duplicates based on specific columns
df_no_dups_subset = drop_duplicates(df_filled, subset=['job_category', 'education'])
print("After dropping duplicates based on job_category and education:", df_no_dups_subset.shape)

# Keep last occurrence instead of first
df_no_dups_last = drop_duplicates(df_filled, subset=df_filled.columns[1:], keep='last')
print("After dropping duplicates (keeping last occurrence):", df_no_dups_last.shape)

In [ ]:
# Let's find and examine a duplicate
duplicated_rows = df_filled[df_filled.duplicated(subset=df_filled.columns[1:], keep=False)]
print(f"Found {len(duplicated_rows)} rows that are duplicates of others")

if len(duplicated_rows) > 0:
    # Find an example duplicate pair
    dup_values = duplicated_rows.iloc[0][df_filled.columns[1:]].values
    example_dups = df_filled[
        (df_filled[df_filled.columns[1:]] == dup_values).all(axis=1)
    ]
    
    print("\nExample of duplicate records:")
    print(example_dups)

For the rest of our analysis, we'll use the dataset with duplicates removed:

In [ ]:
# Use the dataset with duplicates removed
df_clean = df_no_dups.copy()
print(f"Clean dataset shape: {df_clean.shape}")

## 3. ScaleNPipe: Feature Scaling and Model Pipeline

ScaleNPipe helps create scikit-learn pipelines with feature scaling followed by your model.

In [ ]:
# Let's prepare a simple example for binary classification
# We'll predict if a person's income is above the median
median_income = df_clean['income'].median()
df_clean['high_income'] = (df_clean['income'] > median_income).astype(int)

# Select features and target
features = ['age', 'satisfaction', 'years_experience']
X = df_clean[features].values
y = df_clean['high_income'].values

# Check the available scalers
scalers = get_available_scalers()
print("Available scalers:")
for name, desc in scalers.items():
    print(f"- {name}: {desc}")

In [ ]:
# Create a simple model
model = LogisticRegression(random_state=42)

# Create and compare pipelines with different scalers
scaler_types = ['standard', 'minmax', 'robust']
cv_results = {}

for scaler_type in scaler_types:
    # Create pipeline
    pipeline = scale_pipeline(model, scaler=scaler_type)
    
    # Cross-validation
    scores = cross_val_score(pipeline, X, y, cv=5, scoring='accuracy')
    cv_results[scaler_type] = scores
    
    print(f"{scaler_type} scaler - Mean CV accuracy: {scores.mean():.4f}, std: {scores.std():.4f}")

In [ ]:
# Visualize cross-validation results
plt.figure(figsize=(10, 6))

data_to_plot = [cv_results[scaler] for scaler in scaler_types]
plt.boxplot(data_to_plot, labels=scaler_types)

plt.title('Cross-Validation Accuracy with Different Scalers')
plt.xlabel('Scaler Type')
plt.ylabel('Accuracy')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

### Feature Importance Analysis

Let's see how different scaling methods affect the feature coefficients of our logistic regression model:

In [ ]:
# Train pipelines with different scalers and analyze feature coefficients
feature_coefficients = {}

for scaler_type in scaler_types:
    # Create and train pipeline
    pipeline = scale_pipeline(model, scaler=scaler_type)
    pipeline.fit(X, y)
    
    # Extract coefficients
    coefficients = pipeline.named_steps['model'].coef_[0]
    feature_coefficients[scaler_type] = coefficients
    
    print(f"\n{scaler_type.capitalize()} Scaler - Feature Coefficients:")
    for feature, coef in zip(features, coefficients):
        print(f"  {feature}: {coef:.4f}")

In [ ]:
# Visualize feature coefficients
plt.figure(figsize=(12, 6))

x = np.arange(len(features))
width = 0.25

for i, scaler_type in enumerate(scaler_types):
    plt.bar(x + (i-1)*width, feature_coefficients[scaler_type], width, label=scaler_type)

plt.xlabel('Features')
plt.ylabel('Coefficient Value')
plt.title('Feature Coefficients with Different Scalers')
plt.xticks(x, features)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## Summary

In this notebook, we've demonstrated the three main components of the PreProPy package:

1. **NullSense**: We filled missing values using different strategies, with the 'auto' strategy being particularly useful as it applies appropriate methods based on column types.

2. **DupliChecker**: We identified and removed duplicate records, with options for which duplicates to keep and which columns to consider.

3. **ScaleNPipe**: We created scikit-learn pipelines with different scalers, showing how scaling can affect model performance and feature coefficients.

These tools can be easily incorporated into your data science workflow to streamline the preprocessing phase of your projects.